In [1]:
import os,gc
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 

from functools import partial
import albumentations as albu

from sklearn.metrics import roc_auc_score

from kaggle_datasets import KaggleDatasets 
from sklearn.model_selection import train_test_split 
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow.keras.applications.efficientnet as efn 
from tensorflow.keras.applications import ResNet152

In [2]:
def auto_select_accelerator():
    tpu_ok = False 
    try:
        tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
        tf.config.experimental_connect_to_cluster(tpu)
        tf.tpu.experimental.initialize_tpu_system(tpu)
        strategy = tf.distribute.experimental.TPUStrategy(tpu)
        tpu_ok = True
        print("Running on TPU:", tpu.master())
    except ValueError:
        strategy = tf.distribute.get_strategy()
    print(f"Running on {strategy.num_replicas_in_sync} replicas")
    return strategy,tpu_ok

AUTO = tf.data.experimental.AUTOTUNE
strategy,tpu_ok = auto_select_accelerator()
if tpu_ok:
    load_dir = KaggleDatasets().get_gcs_path("ranzcr-clip-catheter-line-classification")
else:
    load_dir = "../input/ranzcr-clip-catheter-line-classification"

Running on 1 replicas


In [3]:
labels = np.array(['ETT - Abnormal', 'ETT - Borderline',
       'ETT - Normal', 'NGT - Abnormal', 'NGT - Borderline',
       'NGT - Incompletely Imaged', 'NGT - Normal', 'CVC - Abnormal',
       'CVC - Borderline', 'CVC - Normal', 'Swan Ganz Catheter Present'])

DEBUG = False 
class CONFIG:
    version = 4 
    batchsize =16*strategy.num_replicas_in_sync
    epochs = 1 if DEBUG else 40    
    imsize = (512,512) 
    shuffle = 2048
    seed = 123 
    lr = 1e-3 
    min_lr = 5e-6
    max_lr = 1e-3
    n_labels = 11 
    n_folds = 5
    tta = 2

print(f"EPOCHS : {CONFIG.epochs}")
print(f"BATCHSIZE : {CONFIG.batchsize}")

EPOCHS : 40
BATCHSIZE : 16


In [4]:
## Utils 
def seed_everything(seed):
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
    
## decoder 
def decoder_image(img):
    img = tf.io.decode_jpeg(img,channels=3)
    img = tf.cast(img,tf.float32) #float32にcast
    img /= 255.0 
    img = tf.image.resize(img,CONFIG.imsize)
    return img


## Augmentation
def augmenter(img,label):
    img = tf.image.random_flip_left_right(img)
    img = tf.image.random_flip_up_down(img)
    img = tf.image.random_hue(img,max_delta=0.01)
    img = tf.image.random_saturation(img,lower=0.70,upper=1.30) #factor
    img = tf.image.random_contrast(img,lower=0.80,upper=1.20) #factor
    img = tf.image.random_brightness(img,max_delta=0.10)
    return img,label

## For TFRecord 
def read_labeled_tfrecord(example):
    LABELED_TFREC_FORMAT = {
        "image" : tf.io.FixedLenFeature([],tf.string),
        "ETT - Abnormal":tf.io.FixedLenFeature([],tf.int64),
        "ETT - Borderline":tf.io.FixedLenFeature([],tf.int64),
        "ETT - Normal":tf.io.FixedLenFeature([],tf.int64),
        "NGT - Abnormal":tf.io.FixedLenFeature([],tf.int64),
        "NGT - Borderline":tf.io.FixedLenFeature([],tf.int64),
        "NGT - Incompletely Imaged":tf.io.FixedLenFeature([],tf.int64),
        "NGT - Normal":tf.io.FixedLenFeature([],tf.int64),
        "CVC - Abnormal":tf.io.FixedLenFeature([],tf.int64),
        "CVC - Borderline":tf.io.FixedLenFeature([],tf.int64),
        "CVC - Normal":tf.io.FixedLenFeature([],tf.int64),
        "Swan Ganz Catheter Present":tf.io.FixedLenFeature([],tf.int64),
        'StudyInstanceUID': tf.io.FixedLenFeature([],tf.string),
        'PatientID':tf.io.FixedLenFeature([],tf.string) 
    }
    example = tf.io.parse_single_example(example,LABELED_TFREC_FORMAT)
    image = decoder_image(example["image"])
    targets = tf.stack([tf.cast(example[label],tf.float32) for label in labels])
    return image,targets

def make_dataset(paths,cache_dir=False,augment=False,repeat=False,shuffle=0):
    if cache_dir:
        os.makedirs(cache_dir,exist_ok=True)
    dset = tf.data.TFRecordDataset(paths)
    dset = dset.map(read_labeled_tfrecord,num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache_dir else dset 
    dset = dset.map(augmenter,num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset 
    dset = dset.batch(CONFIG.batchsize)
    dset = dset.prefetch(AUTO)
    return dset

def change_shape(img_label,teacher):
    img = img_label[0]
    label = img_label[1]
    return img,tf.concat((label,teacher),axis=0)

def make_teacher_dataset(paths,teacher_pred,cache_dir=False,augment=False,repeat=False,shuffle=0):
    dset = tf.data.TFRecordDataset(paths)
    teacher = tf.data.Dataset.from_tensor_slices(teacher_pred)
    dset = dset.map(read_labeled_tfrecord,num_parallel_calls=AUTO)
    dset = tf.data.Dataset.zip((dset,teacher))
    dset = dset.map(change_shape,num_parallel_calls=AUTO)
    
    dset = dset.map(augmenter,num_parallel_calls=AUTO) if augment else dset 
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset 
    dset = dset.batch(CONFIG.batchsize)
    dset = dset.prefetch(AUTO)
    return dset

In [5]:

## Custom Loss
pos_weight = tf.constant([0.99737393,0.96217133,0.75933251,0.99072566,0.98241532,
              0.90865273,0.84054117,0.89379384,0.71877805,0.29116112,
              0.97240967],tf.float32)
neg_weight = tf.constant([0.00262607,0.03782867,0.24066749,0.00927434,0.01758468,
              0.09134727,0.15945883,0.10620616,0.28122195,0.70883888,
              0.02759033],tf.float32)
mul = tf.multiply
def WeightedBinaryCrossentropy(y_true,y_pred):
    y_true = tf.cast(y_true,tf.float32)
    pos_loss = -mul(mul(      y_true,tf.math.log(    y_pred + 1e-13)),pos_weight) 
    neg_loss = -mul(mul(1.0 - y_true,tf.math.log(1.0 - y_pred + 1e-13)),neg_weight) 
    loss = tf.add(pos_loss,neg_loss)
    return tf.math.reduce_mean(loss,0)

def CustomLoss(y_true,y_pred):
    student_feature = y_pred[:,:11]
    student_prob = y_pred[:,11:]
    teacher_feature = y_true[:,11:]
    teacher_prob = y_true[:,:11] 
    
    # BCE 
    pos_loss = -mul(mul(      teacher_prob,tf.math.log(      student_prob + 1e-13)),pos_weight) 
    neg_loss = -mul(mul(1.0 - teacher_prob,tf.math.log(1.0 - student_prob + 1e-13)),neg_weight) 
    loss = tf.add(pos_loss,neg_loss)
    bce = tf.math.reduce_mean(loss,0)
    
    # L2 
    mse = tf.math.square(teacher_feature - student_feature)
    mse = tf.math.reduce_mean(mse,0)
    
    total_loss = tf.math.reduce_mean(bce + mse/2,0) 
    return total_loss


## Metrics
def mean_roc_auc(targets,probabilities):
    roc_auc = [roc_auc_score(targets[:,k],probabilities[:,k]) for k in range(CONFIG.n_labels)]
    return np.average(roc_auc)

metric_fn = tf.keras.metrics.AUC(multi_label=True)
def mean_roc_auc_(y_true,y_pred):
    y_pred = y_pred[:,11:] 
    y_true = y_true[:,:11]
    metric_fn.update_state(y_true,y_pred)
    return metric_fn.result()

## Annealing 
def CosineAnnealing(epoch,lr):
    return CONFIG.min_lr + (CONFIG.max_lr - CONFIG.min_lr)*(1 + np.cos(epoch/CONFIG.epochs*np.pi))/2 


## Model 
def create_teacher_model():
    with strategy.scope():
        model = tf.keras.Sequential([
            ResNet152(input_shape=(*CONFIG.imsize,3),
                                  weights=None, 
                                  include_top=False),
            tf.keras.layers.GlobalAveragePooling2D(),
            tf.keras.layers.Dense(CONFIG.n_labels)
        ])
    return model
    
def create_model():
    with strategy.scope():
        inp = tf.keras.layers.Input(shape=(*CONFIG.imsize,3))
        x = ResNet152(weights="imagenet",include_top=False)(inp)
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        x = tf.keras.layers.Dense(CONFIG.n_labels)(x) 
        y = tf.keras.layers.Activation("sigmoid")(x) 
        out = tf.keras.layers.Concatenate(axis = 1)([x,y])
        model = tf.keras.models.Model(inputs = inp,outputs = out) 
        model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=CONFIG.lr),
                      loss=CustomLoss,
                      metrics=mean_roc_auc_) 
    return model

In [6]:
def cv_tuner_from_tf(annot_path,path,df,n_folds=CONFIG.n_folds):
    scores = []
    seed_everything(CONFIG.seed)
    ANNOT_GCS_PATH = KaggleDatasets().get_gcs_path(annot_path) 
    GCS_PATH = KaggleDatasets().get_gcs_path(path) 
    for fold in range(n_folds):
        print("-"*50)
        if fold == 1: #5fold is time consuming 
            break 
        annot_train_paths = [ANNOT_GCS_PATH + f"/train_annot_{CONFIG.imsize[0]}_{i}.tfrec" for i in range(n_folds) if i != fold]
        annot_valid_path = ANNOT_GCS_PATH + f"/train_annot_{CONFIG.imsize[0]}_{fold}.tfrec"
        train_paths = [GCS_PATH + f"/train_withoutannot_clahe_{CONFIG.imsize[0]}_{i}.tfrec" for i in range(n_folds) if i != fold]
        valid_path = GCS_PATH + f"/train_withoutannot_clahe_{CONFIG.imsize[0]}_{fold}.tfrec"
        
        if tpu_ok:
            annot_train_dset = make_dataset(annot_train_paths) 
            annot_valid_dset = make_dataset(annot_valid_path)  
        else:
            annot_train_dset = make_dataset(annot_train_paths,cache_dir="kaggle_tf_cache")
            annot_valid_dset = make_dataset(annot_valid_path,cache_dir="kaggle_tf_cache")

        # Callbacks 
        ckp_path = f"model_nb11_{CONFIG.version}_{fold}.h5"
        ckp = tf.keras.callbacks.ModelCheckpoint(ckp_path,save_best_only=True,monitor="val_loss",mode="min")
        rlr = tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss",patience=2,factor=0.5,min_lr=1e-6,mode="min")
        #rlr = tf.keras.callbacks.LearningRateScheduler(CosineAnnealing) 
        
        # Training
        # Teacher pred
        print("Teacher Pred...")
        if DEBUG:
            teacher_train_pred = tf.convert_to_tensor(np.zeros((annot_size[fold],11)),dtype=tf.float32)
            teacher_valid_pred = tf.convert_to_tensor(np.zeros((9095-annot_size[fold],11)),dtype=tf.float32)
        else:
            teacher_model = create_teacher_model()
            teacher_model.load_weights(f"../input/ranzcrstageweight/model_nb10_3_{fold}.h5")
            steps = (annot_size[fold] + CONFIG.batchsize - 1)//CONFIG.batchsize 
            teacher_train_pred = teacher_model.predict(annot_train_dset,
                                                       steps = steps, 
                                                       batch_size = CONFIG.batchsize,
                                                       verbose = 1)[:annot_size[fold]]
            steps = (9095 - annot_size[fold] + CONFIG.batchsize - 1)//CONFIG.batchsize 
            teacher_valid_pred = teacher_model.predict(annot_valid_dset,
                                                       steps = steps,
                                                       batch_size = CONFIG.batchsize,
                                                       verbose = 1)[:9095-annot_size[fold]]
        
        train_dset = make_teacher_dataset(train_paths,teacher_train_pred,augment=True,repeat=True,shuffle=True)
        valid_dset = make_teacher_dataset(valid_path,teacher_valid_pred) 
        
        print("Student Training...")
        # Student
        model = create_model()
        steps_per_epoch = annot_size[fold]/CONFIG.batchsize
        history = model.fit(train_dset,
                            epochs=CONFIG.epochs,
                            batch_size=CONFIG.batchsize,
                            verbose=2 if DEBUG else 1, 
                            steps_per_epoch=steps_per_epoch,
                            callbacks=[ckp,rlr],
                            validation_data = valid_dset)
        pd.DataFrame(history.history).to_csv(f"history{CONFIG.version}_{fold}.csv")
        
        print("Evaluate...")
        # Eval 
        model.load_weights(ckp_path) 
        valid_size = 9095 - annot_size[fold]
        steps = (valid_size + CONFIG.batchsize - 1)//CONFIG.batchsize 
        pred = model.predict(valid_dset,
                             steps = steps,
                             batch_size=CONFIG.batchsize,
                             verbose=1)[:valid_size]
        valid_labels = df[df.fold == fold][labels].values
        auc = mean_roc_auc(valid_labels,pred)
        
        print(f"FOLD : AUC {auc}")
        scores.append(auc)
        del train_dset,valid_dset,model,history,train_paths,valid_path
        gc.collect()

In [7]:
df = pd.read_csv("../input/ranzcr-sgkf-data/train_folds.csv")
annot = pd.read_csv("../input/ranzcr-clip-catheter-line-classification/train_annotations.csv")
annot_df = df[df['StudyInstanceUID'].isin(annot['StudyInstanceUID'].unique())].reset_index(drop=True)
print(annot_df.shape[0])

9095


In [8]:
annot_size = [7232,7366,7226,7299,7257]
#cv_tuner_from_tf("ranzcr512-sgkfannot","ranzcr512withoutannotclahe",annot_df)

In [9]:
"""
teacher_pred = tf.convert_to_tensor(np.zeros((1863,11)),dtype=tf.float32)
paths = "../input/ranzcr512-sgkfannot/train_annot_512_0.tfrec"

def change_shape(img_label,teacher):
    img = img_label[0]
    label = img_label[1]
    return img,tf.concat((label,teacher),axis=0)

def make_teacher_dataset(paths,teacher_pred,cache_dir=False,augment=False,repeat=False,shuffle=0):
    dset = tf.data.TFRecordDataset(paths)
    teacher = tf.data.Dataset.from_tensor_slices(teacher_pred)
    dset = dset.map(read_labeled_tfrecord,num_parallel_calls=AUTO)
    dset = tf.data.Dataset.zip((dset,teacher))
    dset = dset.map(change_shape,num_parallel_calls=AUTO)
    
    dset = dset.map(augmenter,num_parallel_calls=AUTO) if augment else dset 
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset 
    dset = dset.batch(CONFIG.batchsize)
    dset = dset.prefetch(AUTO)
    return dset

dset = make_teacher_dataset(paths,teacher_pred)
print(dset)
"""

'\nteacher_pred = tf.convert_to_tensor(np.zeros((1863,11)),dtype=tf.float32)\npaths = "../input/ranzcr512-sgkfannot/train_annot_512_0.tfrec"\n\ndef change_shape(img_label,teacher):\n    img = img_label[0]\n    label = img_label[1]\n    return img,tf.concat((label,teacher),axis=0)\n\ndef make_teacher_dataset(paths,teacher_pred,cache_dir=False,augment=False,repeat=False,shuffle=0):\n    dset = tf.data.TFRecordDataset(paths)\n    teacher = tf.data.Dataset.from_tensor_slices(teacher_pred)\n    dset = dset.map(read_labeled_tfrecord,num_parallel_calls=AUTO)\n    dset = tf.data.Dataset.zip((dset,teacher))\n    dset = dset.map(change_shape,num_parallel_calls=AUTO)\n    \n    dset = dset.map(augmenter,num_parallel_calls=AUTO) if augment else dset \n    dset = dset.repeat() if repeat else dset\n    dset = dset.shuffle(shuffle) if shuffle else dset \n    dset = dset.batch(CONFIG.batchsize)\n    dset = dset.prefetch(AUTO)\n    return dset\n\ndset = make_teacher_dataset(paths,teacher_pred)\nprint(d

In [10]:
"""
x = np.abs(np.random.rand(100,22))
y = np.random.randint(0,2,(100,22))
label = tf.convert_to_tensor(y,dtype=tf.float32)
pred = tf.convert_to_tensor(x,dtype=tf.float32)
print(f"LABEL {label}\nPRED {pred}")
print(f"LABEL SHAPE : {label.shape}\nPRED SHAPE {pred.shape}")
loss = CustomLoss(label,pred)
print(f"LOSS {loss}")
auc = mean_roc_auc_(label,pred)
print(f"AUC {auc}")
"""

'\nx = np.abs(np.random.rand(100,22))\ny = np.random.randint(0,2,(100,22))\nlabel = tf.convert_to_tensor(y,dtype=tf.float32)\npred = tf.convert_to_tensor(x,dtype=tf.float32)\nprint(f"LABEL {label}\nPRED {pred}")\nprint(f"LABEL SHAPE : {label.shape}\nPRED SHAPE {pred.shape}")\nloss = CustomLoss(label,pred)\nprint(f"LOSS {loss}")\nauc = mean_roc_auc_(label,pred)\nprint(f"AUC {auc}")\n'

In [11]:
"""
model = create_teacher_model()
model.load_weights(f"../input/ranzcrstageweight/model_nb10_3_0.h5")
from keras.utils import plot_model
plot_model(model)
"""

'\nmodel = create_teacher_model()\nmodel.load_weights(f"../input/ranzcrstageweight/model_nb10_3_0.h5")\nfrom keras.utils import plot_model\nplot_model(model)\n'